In [1]:
import gdown
import os
from PIL import Image
import gc

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import average_precision_score


import random
import warnings
warnings.filterwarnings("ignore")



In [2]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
seed_everything()

### MultiClass


In [3]:
from utils.trainer import Trainer
from data.multi_class_build_data import build_dataloader

BATCH_SIZE=4

transfrom = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

train_dataloader, test_dataloader, trainset_len, testset_len, NUM_CLASS = build_dataloader(transfrom, BATCH_SIZE)

In [4]:
criterion=nn.CrossEntropyLoss()

In [ ]:
from densenet import DenseNet
import torch
device = torch.device('cuda')
model = DenseNet(num_classes=NUM_CLASS).to(device)

In [11]:
NAME = 'DenseNet'

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

trainer = Trainer(model , 
                  criterion,
                  device,
                  train_dataloader,
                  test_dataloader,
                  trainset_len,
                  testset_len,
                  optimizer,
                  epochs=2,
                  path_output='multi_class_output/{name}.pt'.format(name=NAME),
                  multi_label=False
                 )

In [ ]:
trainer.training()

In [ ]:
_, _, preds = trainer.val()

prediciton = pd.DataFrame([preds[0].cpu().numpy(), preds[1].cpu().numpy()]).T
prediciton.rename = ['labels', 'predictions']

prediciton.to_csv('multi_class_output/{name}.csv'.format(name=NAME), index=False)

### MultiLabel


In [1]:
import gdown
import os
from PIL import Image
import gc

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, f1_score
import torch.nn.functional as F


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sklearn
from sklearn.metrics import average_precision_score


import random
import warnings
warnings.filterwarnings("ignore")


In [2]:
#import pandas as pd
#
#
#data = pd.read_csv('sample_labels.csv')
#data['lables'] = data['Finding Labels'].str.split('|')
#data['Image Index'] = './sample/images/' + data['Image Index']
#
#chen = []
#for path_img in data['Image Index'].values:
#    img = Image.open(path_img)
#    chen.append(transforms.ToTensor()(img).shape)
#
#bad_img = data.loc[pd.DataFrame(chen)[0] > 1, 'Image Index'].values
#
#for path_img in bad_img:
#    img = plt.imread(path_img)
#    img = img[:, :, 0]
#    img = Image.fromarray(np.uint8(img * 255), 'L')
#    img.save(path_img,"PNG")

In [3]:
if not os.path.exists('multi_label_output'):
    os.mkdir('multi_label_output')
else:
    print('multi_label_output exists')

multi_label_output exists


In [4]:
criterion = nn.BCELoss()

In [5]:
from utils.trainer import Trainer
from data.multi_label_build_data import build_dataloader

BATCH_SIZE=6

transfrom = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomRotation(30),
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5)
])

train_dataloader, test_dataloader, trainset_len, testset_len, NUM_CLASS = build_dataloader(transfrom, BATCH_SIZE)

In [6]:
from densenet import DenseNet
import torch
device = torch.device('cuda')
model = DenseNet(num_classes=NUM_CLASS, num_channels=1).to(device)

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

NAME = 'DenseNet'

trainer = Trainer(model , 
                criterion,
                device,
                train_dataloader,
                test_dataloader,
                trainset_len,
                testset_len,
                optimizer,
                epochs=10,
                path_output='multi_label_output/{name}.pt'.format(name=NAME),
                multi_label=True
                )

In [ ]:
trainer.training()

In [10]:
_, _, preds = trainer.val()

pd.DataFrame(preds[0].cpu().numpy()).to_csv('multi_label_output/true_{name}.csv'.format(name=NAME), index=False)
pd.DataFrame(preds[1].cpu().numpy()).to_csv('multi_label_output/pred_{name}.csv'.format(name=NAME), index=False)